In [1]:
!pip install sentence-transformers langchain langchain-community chromadb datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.7 MB/s eta 

In [2]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
from datasets import load_dataset

ds = load_dataset("booksouls/goodreads-book-descriptions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/317M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/314M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1021106 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'description'],
        num_rows: 1021106
    })
})

In [5]:
data_df = ds['train'].to_pandas()
print(data_df.isna().sum())
data_df

title          0
description    0
dtype: int64


,title,description
0,Good Harbor,"Anita Diamant's international bestseller ""The ..."
1,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",Omnibus book club edition containing the Ladie...
2,Best Friends Forever,Addie Downs and Valerie Adler were eight when ...
3,The Aeneid for Boys and Girls,"Relates in vigorous prose the tale of Aeneas, ..."
4,All's Fairy in Love and War (Avalon: Web of Ma...,"To Kara's astonishment, she discovers that a p..."
...,...,...
1021101,The Spaniard's Blackmailed Bride,"Blackmailed into marriage to save her family, ..."
1021102,Different Breeds,Derek and Blake have an understanding -- they ...
1021103,"This Sceptred Isle, Vol. 10: The Age of Victor...","The award-winning story of Britain, from the a..."
1021104,Sherlock Holmes and the July Crisis,Sir Arthur Conan Doyle is brought back to life...


In [6]:
Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                            model_kwargs={'device': 'cuda'})

<ipython-input-6-78f1b0daa4da>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
REVIEWS_CHROMA_PATH = "chroma_data"

loader = DataFrameLoader(data_df, page_content_column="description")
descriptions = loader.load()

In [ ]:
reviews_vector_db = Chroma.from_documents(
    descriptions, Embeddings, persist_directory=REVIEWS_CHROMA_PATH
)

In [8]:
REVIEWS_CHROMA_PATH = "chroma_books_data"

batch_size = 10000  # Adjust batch size based on memory capacity
for start_idx in range(0, len(data_df), batch_size):
    batch = data_df[start_idx:start_idx + batch_size]
    loader = DataFrameLoader(batch, page_content_column="description")
    descriptions = loader.load()
    reviews_vector_db = Chroma.from_documents(
        descriptions, Embeddings, persist_directory=REVIEWS_CHROMA_PATH
    )
    reviews_vector_db.persist()  # Save incrementally


<ipython-input-8-b254ebdd5ee5>:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  reviews_vector_db.persist()  # Save incrementally


In [9]:
question = """
book talking about money and how to become rich
"""

relevant_docs = reviews_vector_db.similarity_search(question, k=3)

In [12]:
for i in range(3):  # Loop over the top 3 suggested books
    title = relevant_docs[i].metadata["title"]
    print(f"Book {i + 1}:")
    print(f"Title: {data_df[data_df['title'] == title]['title'].iloc[0]}")
    print(f"Description: {data_df[data_df['title'] == title]['description'].iloc[0]}")
    print("\n")

Book 1:
Title: The Science of Getting Rich and The Art of Money Getting
Description: Contained within this volume are two classics on the subject of acquiring wealth. Both books look at the subject from the perspective of the fundamental principles that exist for the successful creation of wealth and money making enterprises. At the same time both works seek to be practical guides to help readers figure out for themselves the way to generating a more financially rewarding lifestyle. The path to wealth begins with a shift in the way that you view your life and work and these two short works will greatly help all who read them on the way to not only a more financially rewarding life but an emotionally wealthy one as well.


Book 2:
Title: The Science of Getting Rich
Description: This book claims there is a true science to becoming wealthy, and reveals it within these pages. From the time it first appeared in 1915 people have used its principles successfully, so there is a time-tested sys

In [13]:
!zip -r chroma_books_data.zip chroma_books_data

  adding: chroma_books_data/ (stored 0%)
  adding: chroma_books_data/chroma.sqlite3 (deflated 57%)
  adding: chroma_books_data/6f355f97-eccd-4475-890b-5b64ca72c581/ (stored 0%)
  adding: chroma_books_data/6f355f97-eccd-4475-890b-5b64ca72c581/link_lists.bin (deflated 60%)
  adding: chroma_books_data/6f355f97-eccd-4475-890b-5b64ca72c581/data_level0.bin (deflated 10%)
  adding: chroma_books_data/6f355f97-eccd-4475-890b-5b64ca72c581/header.bin (deflated 53%)
  adding: chroma_books_data/6f355f97-eccd-4475-890b-5b64ca72c581/index_metadata.pickle (deflated 50%)
  adding: chroma_books_data/6f355f97-eccd-4475-890b-5b64ca72c581/length.bin (deflated 65%)


In [16]:
!ls -lh

total 4.4G
drwxr-xr-x 3 root root 4.0K Oct  7 14:20 chroma_books_data
-rw-r--r-- 1 root root 4.4G Oct  7 15:10 chroma_books_data.zip
drwxr-xr-x 1 root root 4.0K Oct  3 13:24 sample_data


In [17]:
from google.colab import drive
drive.mount('/content/drive')

!cp chroma_books_data.zip /content/drive/MyDrive/

Mounted at /content/drive
